In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

#for visualizing null values
#import missingno as msno

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = "25,10"
plt.rcParams["legend.fontsize"] = 16
plt.rcParams["axes.labelsize"] = 16

#import statmodels: this is another library which has similar funtionas as sklearn
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

#import sklearn important functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix #needed for classification problems
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve #receiver operating charecteristic curve for binary classifier model diagnosis
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

#import python based open-source system for mathematical , science and engineering functions
import scipy 

#ignoring warnings
import warnings
warnings.filterwarnings("ignore")

/Users/ctsuser1/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:

data_dir = Path('output')

full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')
)

In [3]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237797 entries, 0 to 1020
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   timestamp          171179 non-null  datetime64[ns]
 1   annotation         237797 non-null  object        
 2   EEG_Fpz-Cz_uV      237797 non-null  object        
 3   EEG_Pz-Oz_uV       237797 non-null  object        
 4   EOG_horizontal_uV  237797 non-null  object        
 5   subject            66618 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 12.7+ MB


In [4]:
# full_df.describe(include = 'all').transpose()

In [5]:
full_df

,timestamp,annotation,EEG_Fpz-Cz_uV,EEG_Pz-Oz_uV,EOG_horizontal_uV,subject
0,1989-07-21 01:30:00,2,"[24.228571428571424, 30.93333333333333, 32.355...","[9.736263736263734, 2.887912087912086, -4.2307...","[-0.22759462759464258, -0.22759462759464258, 1...",NaN
1,1989-07-21 01:30:30,2,"[42.41269841269841, 26.463492063492062, 17.015...","[2.347252747252745, 8.474725274725273, 11.8989...","[52.119169719169705, 49.84322344322343, 33.001...",NaN
2,1989-07-21 01:31:00,2,"[0.1523809523809522, 5.536507936507936, 4.5206...","[-2.6087912087912106, -2.06813186813187, -3.69...","[5.689865689865674, 3.4139194139193982, 4.7794...",NaN
3,1989-07-21 01:31:30,2,"[-6.044444444444444, -5.841269841269842, 0.965...","[-12.971428571428573, -8.195604395604397, -7.8...","[-26.173382173382187, -26.173382173382187, -48...",NaN
4,1989-07-21 01:32:00,2,"[-10.20952380952381, -7.873015873015873, -11.6...","[7.483516483516481, 5.771428571428569, 5.95164...","[5.23467643467642, 3.869108669108654, 3.413919...",NaN
...,...,...,...,...,...,...
1016,1989-07-13 08:20:00,W,"[14.266178266178276, 11.262515262515274, 12.46...","[5.685714285714295, 5.33113553113554, 10.64981...","[51.90989010989016, 46.101343101343154, 38.633...",NaN
1017,1989-07-13 08:20:30,W,"[51.81196581196583, 52.112332112332126, 40.898...","[11.358974358974368, 10.295238095238105, 10.82...","[128.2507936507937, 125.76141636141641, 132.39...",NaN
1018,1989-07-13 08:21:00,W,"[4.454212454212465, 8.35897435897437, -0.65201...","[7.192673992674002, 2.05128205128206, 3.469597...","[-19.037362637362587, -19.037362637362587, -12...",NaN
1019,1989-07-13 08:21:30,W,"[28.683760683760696, 29.484737484737494, 24.57...","[-0.6080586080585995, -2.5582417582417496, -2....","[37.38852258852264, 25.35653235653241, 25.3565...",NaN


In [6]:
DATAPOINTS_IN_EPOCH = 3000  # 30 sec * 100Hz

def preprocess_data(df):
    '''
        stages are one of: W, R, 1, 2, 3, 4, M (Movement time) and ? (not scored)
    '''
    print("Starting with {} epochs".format(len(df)))
    # combine N3 and N4 into N3 stage
    df.loc[df['annotation'] == '4', 'annotation'] = '3'

    # exclude 'M' and '?' epochs
    df = df.loc[df['annotation'] != '?']
    df = df.loc[df['annotation'] != 'M']
    
    # remove epoches that don't have all 3000 data points
    df = df.loc[df['EEG_Fpz-Cz_uV'].map(len) == DATAPOINTS_IN_EPOCH]
    
    # TODO, not sure yet if this is needed:
    # remove 'W' stages that are more than 30 mibutes from any sleep stage
    
    # encode class label
    class2idx = {
        'R': 0,
        '1': 1,
        '2': 2,
        '3': 3,
        'W': 4,
    }
    df['annotation'] = df['annotation'].replace(class2idx)
    print("Finsihed processing data, {} epochs in total".format(len(df)))
    return df


def get_label_stats(df):
    return df.groupby(['annotation']).agg(['count'])
    
full_df = preprocess_data(full_df)
get_label_stats(full_df)

Starting with 237797 epochs
Finsihed processing data, 237422 epochs in total


,timestamp,EEG_Fpz-Cz_uV,EEG_Pz-Oz_uV,EOG_horizontal_uV,subject
,count,count,count,count,count
annotation,,,,,
0,22019,30012,30012,30012,7993
1,18868,21737,21737,21737,2869
2,59294,77534,77534,77534,18240
3,10639,16136,16136,16136,5497
4,60064,92003,92003,92003,31939


In [7]:
# data_tst = full_df[(full_df.subject == "SC4001E0")][['annotation','EEG_Fpz-Cz_uV']]
data_test = full_df[['annotation','EEG_Fpz-Cz_uV']]

test = data_test.to_numpy()

data_test

,annotation,EEG_Fpz-Cz_uV
0,2,"[24.228571428571424, 30.93333333333333, 32.355..."
1,2,"[42.41269841269841, 26.463492063492062, 17.015..."
2,2,"[0.1523809523809522, 5.536507936507936, 4.5206..."
3,2,"[-6.044444444444444, -5.841269841269842, 0.965..."
4,2,"[-10.20952380952381, -7.873015873015873, -11.6..."
...,...,...
1016,4,"[14.266178266178276, 11.262515262515274, 12.46..."
1017,4,"[51.81196581196583, 52.112332112332126, 40.898..."
1018,4,"[4.454212454212465, 8.35897435897437, -0.65201..."
1019,4,"[28.683760683760696, 29.484737484737494, 24.57..."


In [8]:
df_ann = pd.DataFrame(data_test['annotation'])
df_ann

,annotation
0,2
1,2
2,2
3,2
4,2
...,...
1016,4
1017,4
1018,4
1019,4


In [ ]:
df_eeg = pd.DataFrame(data_test['EEG_Fpz-Cz_uV'].tolist())
df_eeg 

In [ ]:
data = df_ann.reset_index(drop=True).merge(df_eeg.reset_index(drop=True), left_index=True, right_index=True)
data

In [ ]:
# data = df_ann.append(df_eeg)
print(df_ann.shape)
print(df_eeg.shape)
# pd.concat([df_ann, df_eeg], axis=1) ## direct concat can't work; need to reset index as above cell does

In [ ]:
# df1 = data[data['annotation'] == 4].iloc[::2] ## this line was there to remove every other 'w' stage data but later on there wasn't need to do that since the spark took care of that 
# df1 = data
# data_bal = data.drop(df1.index)## this line was there to remove every other 'w' stage data but later on there wasn't need to do that since the spark took care of that 
data_bal = data

In [ ]:
# df1 ##every other "annotation = W or 4", which is going to be deleted; not applicable in full dataset now

In [ ]:
data_bal

In [ ]:
#group the annotation counts to find the classes from balanced dataset

data_bal.groupby(['annotation']).agg(['count'])

In [ ]:
data_bal.info()

In [ ]:
data.info()

In [ ]:
data.isna().sum()

# Exploratory data analysis starts here

##1 univariate analysis:  analysis of single feature

In [ ]:
##annotation values

plt.rcParams["axes.labelsize"] = 25

values = pd.DataFrame({'stages': ['NA','R', 'N1', 'N2', 'N3', 'W']})

ax = sns.distplot(data["annotation"], color = "red");

ax.set(xlabel = 'Sleep Stages', ylabel = 'Data Density')

ax.set_xticklabels(values['stages'], fontsize=20)

In [ ]:
##annotation values from the balanced dataset
plt.rcParams["axes.labelsize"] = 25

values = pd.DataFrame({'stages': ['NA','R', 'N1', 'N2', 'N3', 'W']})

ax = sns.distplot(data["annotation"], color = "green");

ax.set(xlabel = 'Sleep Stages', ylabel = 'Data Density')

ax.set_xticklabels(values['stages'], fontsize=25)

In [ ]:
##EEG sleep signals

ax = sns.distplot(data[0:2999], color = "pink");

ax.set(xlabel = 'Sleep Stage Data Values', ylabel = 'Data Density')

In [ ]:
##EEG sleep signals from balanced dataset

ax = sns.distplot(data_bal[0:2999], color = "blue");

ax.set(xlabel = 'Sleep Stage Data Values', ylabel = 'Data Density')

In [ ]:
# Try histogram for 

plt.hist(data_bal['annotation'], color = 'blue')

# Covarience matrix

In [ ]:
cov = data_bal.cov().transpose()
cov

# Starting Feature Engineering

In [ ]:
# X = data_bal[0:2999]
# X = data_bal[[0,1,2,3,4,5]]
X = data_bal.iloc[:, 1:3001]
y = data_bal[["annotation"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)



In [ ]:

lr = LogisticRegression(solver = 'lbfgs')

lr.fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

## Conclusion of logistic regression with all input variables

The accuracy is really low, therefore, it's better to use other models. 

# Applying Linear Discriminant Analysis

In [ ]:
X = data_bal.iloc[:, 1:3001]
y = data_bal[["annotation"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
lda = LinearDiscriminantAnalysis()

In [ ]:
lda.fit(X_train, y_train)
lda.score(X_test, y_test)

In [ ]:
## ALL EVALUATION MATRIX AND ACCURACY SCORES for LDA model
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

y_pred = lda.predict(X_test)

# Find all accuracy scores

acc = accuracy_score(y_test, y_pred)
per_class_percision = precision_score(y_test, y_pred, average=None) # average='None' to return per-class percision
per_class_recall = recall_score(y_test, y_pred, average=None)
average_f1 = f1_score(y_test, y_pred, average='macro')
per_class_f1 = f1_score(y_test, y_pred, average=None)
kappa = cohen_kappa_score(y_test, y_pred)


# plot the graph where we modify the font size along the axis of the not only the title but data values too and change one data values to other

target_names = ['R', 'N1', 'N2', 'N3/N4', 'W']
labels_names = [0,1,2,3, 4] 

cm = confusion_matrix(y_test, y_pred, labels = labels_names)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=target_names)
disp = disp.plot(cmap=plt.cm.Blues,values_format='g')
plt.show()
report = classification_report(y_test, y_pred, labels=labels_names, target_names=target_names)

print(f"***************ACCURACY SCORE etc USING LDA ALGORITHM*************************")
print(f"\n accuracy_score: {acc}")
print(f"\n average_f1: {average_f1}")
print(f"\n cohen_kappa_score: {kappa}")
print(f"\n confusion_matrix:\n {cm}")
print(f"\n Classification Report: {report}")
print("\n y_test", y_test[:10])
print("\n y_pred-> Predicted values of some sample epochs", y_pred[:10])

## Conclusion of LDA 

LDA accuracy is better than logistic regression but still well below the current state of the art.

# Quadratic Discriminant Analysis

In [ ]:
X = data_bal.iloc[:, 1:3001]
y = data_bal[["annotation"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
qda = QuadraticDiscriminantAnalysis()

In [ ]:
qda.fit(X_train, y_train)
qda.score(X_test, y_test)

In [ ]:
## ALL EVALUATION MATRIX AND ACCURACY SCORES for LDA model
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

y_pred = qda.predict(X_test)

# Find all accuracy scores

acc = accuracy_score(y_test, y_pred)
per_class_percision = precision_score(y_test, y_pred, average=None) # average='None' to return per-class percision
per_class_recall = recall_score(y_test, y_pred, average=None)
average_f1 = f1_score(y_test, y_pred, average='macro')
per_class_f1 = f1_score(y_test, y_pred, average=None)
kappa = cohen_kappa_score(y_test, y_pred)


# plot the graph where we modify the font size along the axis of the not only the title but data values too and change one data values to other

target_names = ['R', 'N1', 'N2', 'N3/N4', 'W']
labels_names = [0,1,2,3, 4] 

cm = confusion_matrix(y_test, y_pred, labels = labels_names)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=target_names)
disp = disp.plot(cmap=plt.cm.Blues,values_format='g')
plt.show()
report = classification_report(y_test, y_pred, labels=labels_names, target_names=target_names)


print(f"***************ACCURACY SCORE etc USING QDA ALGORITHM*************************")
print(f"\n accuracy_score: {acc}")
print(f"\n average_f1: {average_f1}")
print(f"\n cohen_kappa_score: {kappa}")
print(f"\n confusion_matrix:\n {cm}")
print(f"\n Classification Report: {report}")
print("\n y_test", y_test[:10])
print("\n y_pred-> Predicted values of some sample epochs", y_pred[:10])

## Conclusion of QDA 

QDA accuracy is much better than logistic regression and LDA.